In [1]:
import os

os.getcwd()
if '.git' not in os.listdir(): 
    os.chdir('./../../../../')
os.listdir()


['run.sh',
 'requirements.txt',
 'ml',
 '.idea',
 'bot_discord',
 'venv',
 '.git',
 'log',
 '.gitignore']

In [2]:
import datetime


import fasttext
import keras_core as keras
import numpy as np
import tensorflow as tf
from keras_core.models import load_model
from sklearn.model_selection import train_test_split
from tensorboard.plugins.hparams import api as hp
import tensorflow_hub as hub
import tensorflow_text as text


Using TensorFlow backend


2024-07-02 11:01:27.676395: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 11:01:27.691605: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 11:01:27.691638: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-02 11:01:27.701059: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-02 11:01:28.648369: W tensorflow/compiler/tf

## import BERT model

In [3]:
bert_model_name = 'small_bert/bert_en_uncased_L-4_H-512_A-8'
#bert_model_name = 'bert_en_uncased_L-12_H-768_A-12'

map_name_to_handle = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/3',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/3',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-6_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-8_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-512_A-8/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-768_A-12/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_base/2',
    'electra_small':
        'https://tfhub.dev/google/electra_small/2',
    'electra_base':
        'https://tfhub.dev/google/electra_base/2',
    'experts_pubmed':
        'https://tfhub.dev/google/experts/bert/pubmed/2',
    'experts_wiki_books':
        'https://tfhub.dev/google/experts/bert/wiki_books/2',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/talkheads_ggelu_bert_en_base/1',
}

map_model_to_preprocess = {
    'bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'bert_en_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_cased_preprocess/1',
    'small_bert/bert_en_uncased_L-2_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-2_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-2_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-2_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-4_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-4_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-4_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-4_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-6_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-6_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-6_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-6_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-8_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-8_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-8_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-8_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-10_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-10_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-10_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-10_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-12_H-128_A-2':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-12_H-256_A-4':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-12_H-512_A-8':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'small_bert/bert_en_uncased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'bert_multi_cased_L-12_H-768_A-12':
        'https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/1',
    'albert_en_base':
        'https://tfhub.dev/tensorflow/albert_en_preprocess/1',
    'electra_small':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'electra_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'experts_pubmed':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'experts_wiki_books':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
    'talking-heads_base':
        'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1',
}

tfhub_handle_encoder = map_name_to_handle[bert_model_name]
tfhub_handle_preprocess = map_model_to_preprocess[bert_model_name]


print(f'BERT model selected           : {tfhub_handle_encoder}')
print(f'Preprocess model auto-selected: {tfhub_handle_preprocess}')

BERT model selected           : https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Preprocess model auto-selected: https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/1


In [16]:
type(tfhub_handle_preprocess)

str

In [4]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)

2024-07-02 11:01:43.230232: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 11:01:43.600099: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 11:01:43.600148: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 11:01:43.602226: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 11:01:43.602307: I external/local_xla/xla/stream_executor

In [5]:
bert_model = hub.KerasLayer(tfhub_handle_encoder)

In [12]:
text = np.expand_dims("ceci est un test", axis=0)
print(text)
text_encoded = bert_preprocess_model(text)
text_encoded
bert_results = bert_model(text_encoded)
print(f'Loaded BERT: {tfhub_handle_encoder}')
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

['ceci est un test']


2024-07-02 11:10:30.948643: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-07-02 11:10:30.948719: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-07-02 11:10:30.949594: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-07-02 11:10:30.949672: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-07-02 11:10:30.950496: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:190] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2024-07-02

Loaded BERT: https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1
Pooled Outputs Shape:(1, 512)
Pooled Outputs Values:[ 0.9818011   0.89277554  0.23231454  0.01589974  0.1806383   0.9957792
  0.99798256  0.33106488 -0.02497336 -0.9987041  -0.09310168 -0.26330215]
Sequence Outputs Shape:(1, 128, 512)
Sequence Outputs Values:[[-0.42536885 -0.0092841   0.90875703 ... -1.3789577   0.867938
   0.19424458]
 [-0.43141297  0.13562825 -0.30347514 ... -1.0568973   0.5224188
  -0.28293064]
 [-0.66645205 -0.39647132  0.6052308  ... -1.3264837   0.34678963
   0.13612354]
 ...
 [-0.0633131   0.22402436  1.0188394  ... -0.84042495  0.38894445
   0.20893508]
 [ 0.12264241  0.4450482   0.7040574  ... -0.84877586  0.8868514
   0.201817  ]
 [ 0.1116228   0.3624912   0.81944793 ... -0.7872127   0.8446359
   0.12332332]]


## Configurations

In [4]:
params_grid = {
    "model": "mlp",
    "embedder": "bert",
    "embedding_data": "cc.fr.300.bin",
    "dataset": "fr_hf.csv",
    "batch_size": 300
}

In [5]:
data_folder_src = 'ml/ml_core/data/processed/'
ml_core_folder_path = 'ml/ml_core/'
data_path_src = f'{data_folder_src}{params_grid["dataset"]}'
date_str = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
experiment_name = f'deepwoke_{params_grid["embedder"]}_{params_grid["model"]}_{params_grid["dataset"]}_{date_str}'

log_dir = f"{ml_core_folder_path}log/fit/{experiment_name}"
model_weight_dst = f'{ml_core_folder_path}model_weights/{experiment_name}-model.keras'

In [6]:

vector_path = f'{ml_core_folder_path}embedded_vector/{params_grid["embedding_data"]}_{params_grid["dataset"]}'
vector_x_dst = f'{vector_path}.x.npy'
vector_y_dst = f'{vector_path}.y.npy'

# loading dataset

In [7]:
X = np.load(vector_x_dst)
y = np.load(vector_y_dst)

## text to vector

In [21]:
def build_classifier_model(output_bias=None):
    if output_bias is not None:
        output_bias = keras.initializers.Constant(output_bias)
        #print(output_bias)

    text_input = keras.layers.Input(shape=())
    encoder_inputs = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')(text_input)
    #encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    net = keras.layers.Dense(512, activation="relu")(net)
    net = keras.layers.Dropout(0.2)(net)
    net = keras.layers.Dense(1, activation="sigmoid", name='classifier')(net)
    #net = keras.layers.Dense(3, activation="softmax", name='classifier', bias_initializer=output_bias)(net)

    return keras.Model(text_input, net)

In [22]:
model = build_classifier_model()
model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['accuracy'])

ValueError: Exception encountered when calling layer 'preprocessing' (type KerasLayer).

A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

Call arguments received by layer 'preprocessing' (type KerasLayer):
  • inputs=<KerasTensor shape=(None,), dtype=float32, sparse=None, name=keras_tensor_2>
  • training=None

## building and training the model

### callbacks

In [8]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
    monitor='loss'
)


In [9]:

class EarlyStoppingLogging(keras.callbacks.Callback):
    def __init__(self, early_stopping_callback, log_dir):
        super().__init__()
        self.early_stopping = early_stopping_callback
        self.stopped_epoch = 0
        self.writer = tf.summary.create_file_writer(log_dir)

    def on_epoch_end(self, epoch, logs=None):
        if self.early_stopping.stopped_epoch > 0:
            self.stopped_epoch = self.early_stopping.stopped_epoch
            with self.writer.as_default():
                tf.summary.scalar('early_stopping_epoch', self.stopped_epoch, step=epoch)
                self.writer.flush()


In [10]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_logging_callback = EarlyStoppingLogging(early_stopping, log_dir)

2024-07-02 10:23:55.452906: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 10:23:55.496078: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 10:23:55.496114: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 10:23:55.497678: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 10:23:55.497713: I tensorflow/compile

In [11]:
callback = [
    early_stopping,
    tensorboard_callback,
    hp.KerasCallback(log_dir, params_grid),
    early_stopping_logging_callback,
]

### training the model

In [12]:

from ml.ml_core.src.model.model_factory import model_factory

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#model = model_factory(params_grid['model'], 300, 1)
# print(model.summary)

# model = keras.models.Sequential()
# # model.add(keras.layers.Dense(127, input_dim=300, activation='relu'))
# model.add(keras.layers.Dense(63, activation='relu'))
# model.add(keras.layers.Dense(63, activation='relu'))
# model.add(keras.layers.Dense(33, activation='relu'))
# model.add(keras.layers.Dense(0, activation='sigmoid'))
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#print(model.summary)

model.fit(X_train,
          y_train,
          epochs=500,
          batch_size=params_grid['batch_size'],
          callbacks=callback,
          validation_data=(X_test, y_test)
          )

<bound method Model.summary of <Functional name=functional_1, built=True>>
Epoch 1/500


2024-07-02 10:23:57.700603: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xc159130 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-02 10:23:57.700644: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-07-02 10:23:57.724326: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-02 10:23:57.931106: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700


45/49 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6841 - loss: 0.6301

2024-07-02 10:23:59.668129: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


49/49 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - accuracy: 0.6898 - loss: 0.6257 - val_accuracy: 0.7687 - val_loss: 0.5361
Epoch 2/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7648 - loss: 0.5363 - val_accuracy: 0.7687 - val_loss: 0.5097
Epoch 3/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7634 - loss: 0.5099 - val_accuracy: 0.7687 - val_loss: 0.5007
Epoch 4/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7647 - loss: 0.4968 - val_accuracy: 0.7697 - val_loss: 0.4821
Epoch 5/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7646 - loss: 0.4858 - val_accuracy: 0.7833 - val_loss: 0.4735
Epoch 6/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7839 - loss: 0.4628 - val_accuracy: 0.7792 - val_loss: 0.4688
Epoch 7/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7872 - loss: 0.4495 - val_accuracy: 0.7869 - val_loss: 0.4649
Epoch 8/500
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7950 - loss: 0.4469 - val_accuracy: 0.7872 - val_loss: 0.

# save model

In [16]:
model.save(model_weight_dst)

### testing with custom sentence

In [17]:
#model_weight_dst = f'../../model_weights/fasttext_nlp_20240613-131608-model.keras'
new_model = load_model(model_weight_dst)

In [18]:
ft = fasttext.load_model('ml/ml_core/data/embedding_data/cc.fr.300.bin')


from ml.ml_core.src.embedding.embedding_utils import text_to_vector

text_nice = [
    "la bonjour je suis un type sympas",
    "bonjour ça va les gens",
    "j'aime pas trop ce mec",
    "cette meuf est cool",
    "les meuf sont trop cool",
    "les mecs sont des gens stylé",
    "le mec est stylé",
    "nan mais mon manager exagère quand il dit que je suis pas assez productif et que je travail pas assez, franchement il abuse",
    "les hommes cis sont toxiques pour la société car ils imposent leur vision paternaliste",
    "Les hommes sont moins matures que les femmes quand il s'agit de prendre des décisions"

]

text_no_nice = [
    "regarde moi cette connasse",
    "les femme leur place c'est à la cuisine",
    "femme au volant, accident",
    "les femmes on peux pas leur faire confiance",
    "Pour une transphobe son passing est pas ouf",
    "Elle l’ouvre quand même beaucoup celle là pour quelqu’un qu’est pas capable d’éduquer ses enfants",
    "Dis donc, c’est pas parce que t’es mal baisée que tu peux tout te permettre ????",
    "Moi ce que j’aime bien chez les filles comme toi c’est que t’es pas du genre prise de tête",
    "les femmes c'est comme le fromage, quand elles puent c'est qu'elles sont bonnes",
    "Oh les grosses tepus",
    "Faut pas t'étonner si tu te fais violer vu comment t'es habillée",
    "Tu deverais avoir des enfants maintenant avant que ce soit trop tard",
    "Vous seriez une collaboratrice idéal pour notre entreprise, malheureusement nous cherchons quelqu’un disponible sur le long terme, et nous craignons qu’à votre âge des changements familiaux ne vous permettent plus de vous investir autant que besoins"
]
wordsvec_nice = [np.expand_dims(text_to_vector(text, ft), axis=0) for text in text_nice]
wordsvec_no_nice = [np.expand_dims(text_to_vector(text, ft), axis=0) for text in text_no_nice]

print("nice output")
for i in range(0, len(wordsvec_nice)):
    prediction = new_model.predict(wordsvec_nice[i], verbose=False)
    print(f"{text_nice[i]} : {prediction > 0.5}")
    print(f"{prediction}")

print("\n\nnot nice output")
for i in range(0, len(wordsvec_no_nice)):
    prediction = new_model.predict(wordsvec_no_nice[i], verbose=False)
    print(f"{text_no_nice[i]} : {prediction > 0.5}")
    print(f"{prediction}")

nice output
la bonjour je suis un type sympas : [[False]]
[[0.00716593]]
bonjour ça va les gens : [[False]]
[[0.00271189]]
j'aime pas trop ce mec : [[False]]
[[1.9822155e-05]]
cette meuf est cool : [[False]]
[[7.958688e-08]]
les meuf sont trop cool : [[False]]
[[3.9379524e-11]]
les mecs sont des gens stylé : [[False]]
[[1.0103315e-05]]
le mec est stylé : [[False]]
[[3.5561166e-06]]
nan mais mon manager exagère quand il dit que je suis pas assez productif et que je travail pas assez, franchement il abuse : [[False]]
[[0.01031799]]
les hommes cis sont toxiques pour la société car ils imposent leur vision paternaliste : [[ True]]
[[0.9999633]]
Les hommes sont moins matures que les femmes quand il s'agit de prendre des décisions : [[ True]]
[[0.99984384]]


not nice output
regarde moi cette connasse : [[False]]
[[0.02953639]]
les femme leur place c'est à la cuisine : [[False]]
[[0.01907552]]
femme au volant, accident : [[False]]
[[0.00229132]]
les femmes on peux pas leur faire confiance : 